
1. For each interested word/phrase, find the adjectives around this word. Should find the required word inside a sentence.
2. Also generate a X matrix to represent whether the interested word/phrase appears. (1--appear, 0--no appear)

In [1]:
import numpy as np
import pandas as pd
import ast
from textblob import TextBlob
from nltk import pos_tag

In [2]:
## 42 words

words = ["wait", "waiting", "waiter", "waitress", "serve", "service", "services", "staff", "paid", "price","prices",
         "place", "places", "order", "orders", "menu", "lunch",  "people", "table", "tables", "dinner", "times", "flavor", "minutes", "minute",
         "area", "family", "location", "style", "buffet", "friends", "friend", "selection", "pay", "payment",
         "amount", "variety", "pieces", "money", "bill", "server", "servers"]

In [3]:
#24 2-grams phrases
phrases_2 = [('dim','sum'), ('fried','rice'), ('orange', 'chicken'), ('chow', 'mein'), ('pad', 'thai'), ('sour', 'soup'), ('hot', 'pot'), ('spring', 'rolls'), ('lo', 'mein'), ('bbq', 'pork'), ('mongolian', 'beef'), ('egg', 'roll'), ('wonton', 'soup'), ('soy', 'sauce'), ('ice', 'cream'), ('pork', 'belly'), ('milk', 'tea'), ('fried', 'chicken'),  ('sour', 'chicken'), ('white', 'rice'), ('sesame', 'chicken'), ('chicken', 'wings'), ('peking', 'duck'), ('bubble', 'tea')]

In [4]:
#10 -grams phrases
phrases_3 = [('egg', 'drop', 'soup'), ('beef', 'noodle', 'soup'), ('kung', 'pao', 'chicken'), ('chicken', 'fried', 'rice'), ('pork', 'fried', 'rice'), ('black', 'bean', 'sauce'), ('shrimp', 'fried', 'rice'), ('xiao', 'long', 'bao'), ('egg', 'foo', 'young'), ('dan', 'dan', 'noodles')]

Second Start

In [5]:
file = pd.read_csv("Chinese_review.csv")

In [6]:
sentencs_token_by_word = file["sentencs_token_by_word"]

In [7]:
text = file["text"]

In [8]:
import ast
sentencs_token_by_word = sentencs_token_by_word.tolist()
sentencs_token_by_word = [ast.literal_eval(review) for review in sentencs_token_by_word]

In [9]:
reviews_token = file["text_token"].tolist()
reviews_token = [ast.literal_eval(review) for review in reviews_token]

In [10]:
bigram = file["bigram"]
bigram_token = bigram.tolist()
bigram_token = [ast.literal_eval(review) for review in bigram_token]

trigram = file["trigram"]
trigram_token = trigram.tolist()
trigram_token = [ast.literal_eval(review) for review in trigram_token]

In [12]:
## get The matrix for one-word, 1 means the word appears in the review
data_phrases_1 = []

for i in reviews_token:
    score = []
    for phrase in words:
        if phrase in i:
            score.append(1)
        else:
            score.append(0)
    
    data_phrases_1.append(score)

In [13]:
#get The matrix for two-words and three_words, 1 means the word appears in the review
data_phrases_2 = []

for i in bigram_token:
    score = []
    for phrase in phrases_2:
        if phrase in i:
            score.append(1)
        else:
            score.append(0)
    
    data_phrases_2.append(score)
    
data_phrases_3 = []

for i in trigram_token:
    score = []
    for phrase in phrases_3:
        if phrase in i:
            score.append(1)
        else:
            score.append(0)
    
    data_phrases_3.append(score)

In [14]:
X1 = np.array(data_phrases_1)
X2 = np.array(data_phrases_2)
X3 = np.array(data_phrases_3)

Main Code. Fine the adj for singleword/phrase(2 words)/phrase(3 words)

In [ ]:
### Run 2-3 hours

total_number = len(sentencs_token_by_word)

## total_number = 40

required_tag = np.array(["JJ", "JJR", "JJS","RB"])  ## want to find adj for a given noun. Note: tag of "not" is RB

total_adjective_1 = [None] * total_number  ## store the adj for seleted one-word in wach review
total_adjective_2 = [None] * total_number  ## store the adj for seleted two-word in wach review
total_adjective_3 = [None] * total_number  ## store the adj for seleted three-word in wach review
# score_1 = [None] * total_number
# score_2 = [None] * total_number
# score_3 = [None] * total_number


len1 = len(words)
len2 = len(phrases_2)
len3 = len(phrases_3)

for indexReview in range(total_number):
    
    review = sentencs_token_by_word[indexReview]
    NumberOfSentence = len(review)
    
    ## print(review)   
    ## print(indexReview)
    
    if review == [None]:
        total_adjective_1[indexReview] = list()
        total_adjective_2[indexReview] = list()
        total_adjective_3[indexReview] = list()
        continue
        
    ## review_tag = [pos_tag(r) for r in review]  ## get the tag for each word
    
    review_tag = list()  ## Tag for a review
    for r in review:  ## each sentence
        tags = list()  ## tag for a sentence
        for w in r:
            word = list()
            word.append(w)
            tags.append(pos_tag(word)[0])
        review_tag.append(tags)
    
    existWordsIndex1 = np.where(np.array(X1[indexReview,:]) == 1)[0] ## the existed target word in this review
    existWordsIndex2 = np.where(np.array(X2[indexReview,:]) == 1)[0]
    existWordsIndex3 = np.where(np.array(X3[indexReview,:]) == 1)[0]
        
        
    ### part1: Deal with single-word
    
    if len(existWordsIndex1) > 0:
        ## print("have words!!")
        adj = list()  ## a list of vector, each vector represent the adj for a selected single-word
        score = list()  ## record the sentiment score for a sentence where the target word appears
        ## iterate the words
        for i in range(len1):
            ## print("i: ",i)
            adjForWord = []
            ## scoreForWordsSentence = []
            
            if i in existWordsIndex1:
                word = words[i]        
                
                ## print("target word: ", word)
                
                Score = 0
                
                for indexOfSentence in range(NumberOfSentence):
                    sentence = review[indexOfSentence]
                    tag = review_tag[indexOfSentence]

                    if word in sentence:
                        ##print("sentence: ", sentence)
                        
                        ## combine the token to a string, in order to calculate a sentiment score for this sentence
                        sentenceString = ""
                        for w in sentence:
                            sentenceString += w
                            sentenceString += " "
                        
                        Score += TextBlob(sentenceString).sentiment.polarity
                        
                        ## find adj/adv
                        wordIndexArray = np.where(np.array(sentence) == word)[0]  ##  The index of word in one sentence, a word may exist more than one time                  
                        for wordIndex in wordIndexArray:

                            if wordIndex > 0:  ## The word does not appear in the beginning of a sentence, so may exist a adj before this word

                                ## We assume that the adj appears either before the word or after the word                          
                                if tag[wordIndex - 1][1] in required_tag:  ## exists adj before the word
                                    if wordIndex - 2 >= 0:
                                        if tag[wordIndex - 2][1] == "not":
                                            nw = "not " + tag[wordIndex - 1][0]
                                            adjForWord.append(nw)
                                        else: 
                                            adjForWord.append(tag[wordIndex - 1][0])
                                    else:
                                        adjForWord.append(tag[wordIndex - 1][0])
                                        
                                    ## print("adj before the target word: ", adjForWord )
                                else:
                                    ### looking for required adj/verb after the word
                                    j = wordIndex + 1
                                    while j < len(sentence):
                                        if tag[j][1] in required_tag:
                                            adjForWord.append(tag[j][0])
                                            j += 1
                                        else:
                                            break

                            else:  ## The word is in the beginning of the sentence, find the adj after the word
                                j = wordIndex + 1
                                while j < len(sentence):
                                    if tag[j][1] in required_tag:
                                        adjForWord.append(tag[j][0])
                                        j += 1
                                    else:
                                        break

                            ## print("word in this sentence:", adjForWord)
                ## scoreForWordsSentence.append(Score)
                Sscore = "Score: " + str(Score)
                adjForWord.append(Sscore)
                
                ##print("wordForWholeReview: ", adjForWord)
            adj.append(adjForWord)
            ## score.append(scoreForWordsSentence)    
            ## print("adj", adj)
                    
    else:
        
        adj = list()
        ## score = list()
        ##print("adj", adj)
        
    total_adjective_1[indexReview] = adj
    ## score_1[indexReview] = score
    
    
    ### part2: Deal with two-words phrase
    
    if len(existWordsIndex2) > 0:

        adj = list()  ## a list of vector, each vector represent the adj for a selected single-word
        ## score = list()
        ## iterate the words
        for i in range(len2):
            ## print("i: ",i)
            adjForWord = []
            ## scoreForWordsSentence = []
            if i in existWordsIndex2:
                word = phrases_2[i]    ## the format is a tuple
                firstWord = word[0]
                secondWord = word[1]
                
                ##print("2-phrase: ", word)
                Score = 0
                
                for indexOfSentence in range(NumberOfSentence):
                    sentence = review[indexOfSentence]
                    tag = review_tag[indexOfSentence]
                    ## print(sentence)
                    
                    if firstWord in sentence:
                        wordIndex1 = np.where(np.array(sentence) == firstWord)[0][0]  ##  The index of word in one sentence
                        
                        ## check whether the whole phrase is in the sentence:
                        if wordIndex1 + 1 < len(sentence):
                            if sentence[wordIndex1 + 1] != secondWord:
                                continue
                        else:
                            continue

                        ##print("sentence: ", sentence)
                        
                        ## combine the token to a string, in order to calculate a sentiment score for this sentence
                        sentenceString = ""
                        for w in sentence:
                            sentenceString += w
                            sentenceString += " "
                        
                        Score += TextBlob(sentenceString).sentiment.polarity
                        
                        if wordIndex1 > 0:  ## The word does not appear in the beginning of a sentence, so may exist a adj before this word
                            
                            ## We assume that the adj appears either before the word or after the word                          
                            if tag[wordIndex1 - 1][1] in required_tag:  ## exists adj before the word
                                
                                if wordIndex1 - 2 >= 0:
                                    if tag[wordIndex1 - 2][1] == "not":
                                        nw = "not " + tag[wordIndex1 - 1][0]
                                        adjForWord.append(nw)
                                    else: 
                                        adjForWord.append(tag[wordIndex1 - 1][0])
                                else:
                                    adjForWord.append(tag[wordIndex1 - 1][0])
                             
                            else:
                                ### looking for required adj/verb after the phrase
                                j = wordIndex1 + 2
                                while j < len(sentence):
                                    if tag[j][1] in required_tag:
                                        adjForWord.append(tag[j][0])
                                        j += 1
                                    else:
                                        break
                                        
                        else:  ## The phrase is in the beginning of the sentence, find the adj after the word
                            j = wordIndex1 + 2
                            while j < len(sentence):
                                if tag[j][1] in required_tag:
                                    adjForWord.append(tag[j][0])
                                    j += 1
                                else:
                                    break

                            ##print("word in this sentence:", adjForWord)
                
                Sscore = "Score: " + str(Score)
                adjForWord.append(Sscore)
                
                ## scoreForWordsSentence.append(Score)
                ##print("wordForWholeReview: ", adjForWord)
            adj.append(adjForWord)
            ## score.append(scoreForWordsSentence)    
            ## print("adj", adj)
                    
    else:
        
        adj = list()
        ##score = list()
        ##print("adj", adj)
        
    total_adjective_2[indexReview] = adj
    ##score_2[indexReview] = score    
        
        

        
    ### part3: Deal with three-words phrase
    
    if len(existWordsIndex3) > 0:
        
        adj = list()  ## a list of vector, each vector represent the adj for a selected single-word
        ## score = list()
        ## iterate the words
        for i in range(len3):
            ## print("i: ",i)
            adjForWord = []
            ## scoreForWordsSentence = []
            if i in existWordsIndex3:
                word = phrases_3[i]    ## the format is a tuple
                firstWord = word[0]
                secondWord = word[1]
                thirdWord = word[2]
                
                ##print("3-phrase: ", word)
                Score = 0
                
                for indexOfSentence in range(NumberOfSentence):
                    sentence = review[indexOfSentence]
                    tag = review_tag[indexOfSentence]
                    ## print(sentence)
                    
                    if firstWord in sentence:
                        wordIndex1 = np.where(np.array(sentence) == firstWord)[0][0]  ##  The index of word in one sentence
                        
                        ## check whether the whole phrase is in the sentence:
                        if wordIndex1 + 2 < len(sentence):
                            if sentence[wordIndex1 + 1] != secondWord:
                                continue
                            else:
                                if sentence[wordIndex1 + 2] != thirdWord:
                                    continue
                        else:
                            continue
                            
                            
                        ##print("sentence: ", sentence)
                        
                        ## combine the token to a string, in order to calculate a sentiment score for this sentence
                        sentenceString = ""
                        for w in sentence:
                            sentenceString += w
                            sentenceString += " "
                        
                        Score += TextBlob(sentenceString).sentiment.polarity
                        

                        if wordIndex1 > 0:  ## The word does not appear in the beginning of a sentence, so may exist a adj before this word
                            
                            ## We assume that the adj appears either before the word or after the word                          
                            if tag[wordIndex1 - 1][1] in required_tag:  ## exists adj before the word
                                
                                if wordIndex1 - 2 >= 0:
                                    if tag[wordIndex1 - 2][1] == "not":
                                        nw = "not " + tag[wordIndex1 - 1][0]
                                        adjForWord.append(nw)
                                    else: 
                                        adjForWord.append(tag[wordIndex1 - 1][0])
                                else:
                                    adjForWord.append(tag[wordIndex1 - 1][0])
                                    
                                    
                            else:
                                ### looking for required adj/verb after the phrase
                                j = wordIndex1 + 3
                                while j < len(sentence):
                                    if tag[j][1] in required_tag:
                                        adjForWord.append(tag[j][0])
                                        j += 1
                                    else:
                                        break
                                        
                        else:  ## The phrase is in the beginning of the sentence, find the adj after the word
                            j = wordIndex1 + 3
                            while j < len(sentence):
                                if tag[j][1] in required_tag:
                                    adjForWord.append(tag[j][0])
                                    j += 1
                                else:
                                    break

                            ##print("word in this sentence:", adjForWord)
                
                Sscore = "Score: " + str(Score)
                adjForWord.append(Sscore)
                
                ## scoreForWordsSentence.append(Score)
                ##print("wordForWholeReview: ", adjForWord)
            adj.append(adjForWord)
            ## score.append(scoreForWordsSentence)
            ## print("adj", adj)
                    
    else:
        
        adj = list()
        ## score = list()
        ##print("adj", adj)
        
    total_adjective_3[indexReview] = adj
    ## score_3[indexReview] = score

Create a dataframe to store these adjs.

In [ ]:
total_adjective_1_df = pd.DataFrame(total_adjective_1)
total_adjective_1_df.columns = words
total_adjective_1_df["business_index"] = file["business_index"]
total_adjective_1_df["business_id"] = file["business_id"]
total_adjective_1_df['stars'] = file['stars']

In [ ]:
total_adjective_2_df = pd.DataFrame(total_adjective_2)
total_adjective_2_df.columns = phrases_2
total_adjective_2_df["business_index"] = file["business_index"]
total_adjective_2_df["business_id"] = file["business_id"]
total_adjective_2_df['stars'] = file['stars']

In [ ]:
total_adjective_3_df = pd.DataFrame(total_adjective_3)
total_adjective_3_df.columns = phrases_3
total_adjective_3_df["business_index"] = file["business_index"]
total_adjective_3_df["business_id"] = file["business_id"]
total_adjective_3_df['stars'] = file['stars']

In [ ]:
total_adjective_1_df.to_csv("total_adjective_1_df.csv")
total_adjective_2_df.to_csv("total_adjective_2_df.csv")
total_adjective_3_df.to_csv("total_adjective_3_df.csv")

In [15]:
## Read in the file after close

total_adjective_1_df = pd.read_csv("total_adjective_1_df.csv")
total_adjective_2_df = pd.read_csv("total_adjective_2_df.csv")
total_adjective_3_df = pd.read_csv("total_adjective_3_df.csv")

In [16]:
total_adjective_1_df  = total_adjective_1_df.iloc[:,1:]
total_adjective_2_df  = total_adjective_2_df.iloc[:,1:]
total_adjective_3_df  = total_adjective_3_df.iloc[:,1:]

In [ ]:
### the total number of adjs for each selected word/phrase

count_1 = [0] * 42
count_2 = [0] * 24
count_3 = [0] * 10

for i in range(total_number):
    for j1 in range(42):
        if total_adjective_1_df.iloc[i,j1] != None and type(total_adjective_1_df.iloc[i,j1]) == list:
            length1 = len(total_adjective_1_df.iloc[i,j1])
            if length1 > 0:
                count_1[j1] += length1 - 1
    for j2 in range(25):
        if total_adjective_2_df.iloc[i,j2] != None and type(total_adjective_2_df.iloc[i,j2]) == list:
            length2 = len(total_adjective_2_df.iloc[i,j2])
            if length2 > 0:
                count_2[j2] += length2 - 1
    for j3 in range(10):
        if total_adjective_3_df.iloc[i,j3] != None and type(total_adjective_3_df.iloc[i,j3]) == list:
            length3 = len(total_adjective_3_df.iloc[i,j3])
            if length3 > 0:
                count_3[j3] += length3 - 1

In [ ]:
wordAdjCount1_df = pd.DataFrame()
wordAdjCount1_df["words"] = words
wordAdjCount1_df["adjCounts"] = count_1

In [ ]:
wordAdjCount1_df

In [ ]:
wordAdjCount2_df = pd.DataFrame()
wordAdjCount2_df["words"] = phrases_2
wordAdjCount2_df["adjCounts"] = count_2

In [ ]:
wordAdjCount2_df

In [ ]:
wordAdjCount3_df = pd.DataFrame()
wordAdjCount3_df["words"] = phrases_3
wordAdjCount3_df["adjCounts"] = count_3

In [ ]:
wordAdjCount3_df

In [ ]:
### split the adj for each business. Do not run it again. No use in final report

total_number = file.shape[0]

for i in range(total_number):  
    indexRange = np.where(total_adjective_1_df["business_index"] == i)[0]
    temp1 = total_adjective_1_df.iloc[indexRange,0:-3]
    temp2 = total_adjective_2_df.iloc[indexRange,0:-3]
    temp3 = total_adjective_3_df.iloc[indexRange,0:-3]
#     total_adjective = pd.concat( [temp1.reset_index(drop=True), 
#                                   temp2.reset_index(drop=True), temp3], axis=1)
    
    total_adjective = temp1
    total_adjective[phrases_2] = temp2
    total_adjective[phrases_3] = temp3
    total_adjective["business_index"] = file["business_index"][indexRange]
    total_adjective["business_id"] = file["business_id"][indexRange]
    total_adjective['stars'] = file['stars'][indexRange]
    name = "eachBusiness/business_" + str(i) + ".csv"
    total_adjective.to_csv(name)

In [19]:
index = range(134,139)
X11 = X1[index,:]
X12 = X2[index, :]
X13 = X3[index, :]
X1_ = np.hstack((X11,X12,X13))

In [24]:
stars = np.array(file["stars"])
Y1_ = stars[index]

In [36]:
all_words = words + phrases_2 + phrases_3  ## all words

In [60]:
total_number = X1.shape[0]

In [71]:
## Combine the adjs for single-noun word, stored in Words1

Words1 = [None] * 42
SumOfScore1 = [0] * 42

for j in range(42):
    adjs = list()
    SCORE = 0
    for i in range(total_number):
        ## print(total_adjective_1_df.iloc[i,j])
        if type(total_adjective_1_df.iloc[i,j]) != str:
            continue
            
        item = ast.literal_eval(total_adjective_1_df.iloc[i,j])

        if item != None and item != []:
            if len(item) > 1:
                word = item[0:-1]
                
                for w in word:
                    adjs.append(w)
                
                score = item[-1]
            else:
                score = item[0]
                
            score = float(score[7:])
            SCORE += score
                
    Words1[j] = adjs
    SumOfScore1[j] = SCORE


In [69]:
Words1

[['short',
  'bad',
  '20-30',
  'not',
  'least',
  'not',
  'really',
  'quite',
  'especially',
  'long',
  'long',
  'prepared',
  'never',
  'long',
  'not',
  'even',
  'not'],
 ['long', 'next'],
 ['first', 'not', 'often', 'actually'],
 ['sometimes'],
 ['not', 'green'],
 ['average',
  'slightly',
  'average',
  'poor',
  'average',
  'huge',
  'horrible',
  'pretty',
  'good',
  'poor',
  'terrible',
  'good',
  'not',
  'great',
  'non-dim',
  'poor',
  'poor',
  'bad',
  'honestly',
  'horrible',
  'good',
  'busy',
  'incredibly',
  'clearly',
  'not',
  'interested',
  'good',
  'good',
  'poor',
  'worst',
  'bad',
  'bad',
  'sometimes',
  'bad',
  'average',
  'attentive',
  'poor',
  'back',
  'generally',
  'terrible',
  'horrible',
  'worst',
  'also',
  'exceptionally',
  'bad',
  'expensive',
  'good',
  'however',
  'not',
  'good',
  'good',
  'bad',
  'also',
  'snappy',
  'terrible'],
 [],
 ['delicious', 'constantly', 'friendly', 'not'],
 ['slightly'],
 ['high', '

In [72]:
## Combine the adjs for two-noun word, stored in Words2

Words2 = [None] * 24
SumOfScore2 = [0] * 24

for j in range(24):
    adjs = list()
    SCORE = 0
    for i in range(total_number):
        if type(total_adjective_2_df.iloc[i,j]) != str:
            continue
            
        item = ast.literal_eval(total_adjective_2_df.iloc[i,j])

        if item != None and item != []:
            if len(item) > 1:
                word = item[0:-1]
                
                for w in word:
                    adjs.append(w)
                
                score = item[-1]
            else:
                score = item[0]
                
            score = float(score[7:])
            SCORE += score
                
    Words2[j] = adjs
    SumOfScore2[j] = SCORE


In [73]:
## Combine the adjs for there-noun word, stored in Words1


Words3 = [None] * 10
SumOfScore3 = [0] * 10

for j in range(10):
    adjs = list()
    SCORE = 0
    for i in range(total_number):
        ## print(total_adjective_1_df.iloc[i,j])
        if type(total_adjective_3_df.iloc[i,j]) != str:
            continue
            
        item = ast.literal_eval(total_adjective_3_df.iloc[i,j])

        if item != None and item != []:
            if len(item) > 1:
                word = item[0:-1]
                
                for w in word:
                    adjs.append(w)
                
                score = item[-1]
            else:
                score = item[0]
                
            score = float(score[7:])
            SCORE += score
                
    Words3[j] = adjs
    SumOfScore3[j] = SCORE


In [74]:
WordsTotal = Words1 + Words2 + Words3
SumOfScore = SumOfScore1 + SumOfScore2 + SumOfScore3

In [82]:
## Total number of appearce for adjs.

numberOfAppear = np.hstack((np.sum(X1, 0), np.sum(X2, 0), np.sum(X3, 0)))

In [84]:
AverageSumOfScore = [0] * 76
for i in range(76):
    AverageSumOfScore[i] = SumOfScore[i] / numberOfAppear[i]

In [115]:
WordsTotalt = pd.DataFrame(np.array(WordsTotal))

In [118]:
WordsTotalt["AverageSumOfScore"] = np.array(AverageSumOfScore)

In [120]:
WordsTotalt.to_csv("WordsTotalt.csv")

In [121]:
from collections import Counter

In [131]:
## Most Common adjs for "waiting"

## wait/waiting(0,1) 
WaitWord = WordsTotal[0] + WordsTotal[1]
d_wait = Counter(WaitWord)  

In [132]:
d_wait.most_common(20)

[('not', 4366),
 ('long', 2087),
 ('still', 517),
 ('never', 415),
 ('prepared', 385),
 ('short', 206),
 ('little', 180),
 ('even', 174),
 ('good', 166),
 ('busy', 160),
 ('always', 141),
 ('bad', 120),
 ('least', 113),
 ('sometimes', 107),
 ('much', 106),
 ('usually', 104),
 ('forever', 104),
 ('also', 100),
 ('ready', 99),
 ('almost', 96)]

In [145]:
## Most Common adjs for "service"

WaiterWord = WordsTotal[2] + WordsTotal[3] + WordsTotal[7] + WordsTotal[4] + WordsTotal[5] + WordsTotal[6] + WordsTotal[40] + WordsTotal[41]
d_waiter = Counter(WaiterWord) 

In [146]:
d_waiter.most_common(20)

[('friendly', 13903),
 ('good', 11905),
 ('great', 11848),
 ('not', 7454),
 ('nice', 4585),
 ('attentive', 4151),
 ('always', 3281),
 ('bad', 2912),
 ('also', 2388),
 ('really', 2336),
 ('horrible', 1783),
 ('terrible', 1714),
 ('pretty', 1640),
 ('poor', 1375),
 ('even', 1210),
 ('best', 1117),
 ('well', 1044),
 ('however', 868),
 ('extremely', 858),
 ('chinese', 854)]

In [147]:
## Most Common adjs for "price"

PayWord = WordsTotal[8] + WordsTotal[9] + WordsTotal[10] + WordsTotal[33] + WordsTotal[38] + WordsTotal[39]
d_Pay = Counter(PayWord)

In [143]:
d_Pay.most_common()

[('reasonable', 5615),
 ('good', 4662),
 ('great', 3501),
 ('not', 2955),
 ('high', 1276),
 ('low', 943),
 ('affordable', 830),
 ('higher', 798),
 ('little', 766),
 ('also', 654),
 ('extra', 635),
 ('back', 625),
 ('average', 602),
 ('bad', 576),
 ('pretty', 570),
 ('much', 569),
 ('really', 520),
 ('especially', 467),
 ('expensive', 446),
 ('even', 445),
 ('small', 441),
 ('full', 421),
 ('total', 401),
 ('well', 383),
 ('lower', 316),
 ('still', 294),
 ('chinese', 278),
 ('definitely', 272),
 ('regular', 267),
 ('special', 256),
 ('large', 244),
 ('friendly', 243),
 ('however', 220),
 ('big', 216),
 ('quite', 213),
 ('huge', 209),
 ('nice', 205),
 ('already', 197),
 ('overall', 192),
 ('never', 191),
 ('best', 190),
 ('delicious', 185),
 ('double', 164),
 ('willing', 161),
 ('generous', 157),
 ('first', 156),
 ('rather', 151),
 ('enough', 148),
 ('similar', 146),
 ('different', 143),
 ('always', 141),
 ('ridiculous', 139),
 ('extremely', 138),
 ('comparable', 135),
 ('slightly', 134)